<a href="https://colab.research.google.com/github/Shreyash007/deep_learning_course/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1 CS6910**
Shreyash Gadgil (ED22S016)

In [1]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a735c89b6c2b3c4c6a2da42da5f3d6de0f123597ee90e31b63c7947f344ee075
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
wandb: A

Import libraries

In [12]:
from keras.datasets import fashion_mnist
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import wandb

Q1 SOLUTION
Labelling dataset

In [3]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
#importing dataset

#checking size of training and test dataset
print("Training dataset shape, X=",X_train.shape,", Y=",Y_train.shape)
print("Test dataset shape, X=",X_test.shape,", Y=",Y_test.shape)

n_img_classes=len(np.unique(Y_train)) #to find out number of unique images 
img_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation


def show_images(n_img_classes,img_labels,X,Y):

   setImgs = []
   setLabels=[]

   for i in range(len(X)):
     if len(setImgs)==n_img_classes:
       break
     if img_labels[Y[i]] not in setLabels:
      setImgs.append(X[i])
      setLabels.append(img_labels[Y[i]])

   run=wandb.init(project='CS-6910 A1',entity='shreyashgadgil007',reinit=True)
   run.log({"Fashion_MNIST dataset":list(wandb.Image(img,caption= caption) for img,caption in zip(setImgs,setLabels))})
   run.finish()

#---------------------------------------Q1 SOLUTION-------------------------------------------------------------------
show_images(n_img_classes,img_labels,X_test,Y_test)


4422102/4422102 [==============================] - 0s 0us/step
Training dataset shape, X= (60000, 28, 28) , Y= (60000,)
Test dataset shape, X= (10000, 28, 28) , Y= (10000,)


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shreyashgadgil007. Use `wandb login --relogin` to force relogin


# **Preprocessing dataset for operations**
1. Creating training,test,validation split for dataset
2. Reshaping image matrix and normalizing pixel values
3. One hot encoding classifications


In [ ]:
def dataset_preprocess():
  #change the code below to accept different dataset
  (X_1, Y_1), (X_test, Y_test) = fashion_mnist.load_data()
  
  #importing dataset again and normalizing
  X_1 = (X_1.reshape(X_1.shape[0],-1))/255.0
  X_test = (X_test.reshape(X_test.shape[0],-1))/255.0

  #training and validation split as specified in the question 10%
  X_train, X_val, Y_train, Y_val= train_test_split(X_1,Y_1,test_size=0.1,random_state=0)
  
  #one hot encoding
  Y_train_encoded=np.zeros((Y_train.shape[0],10))
  for i in range(len(Y_train)):
    Y_train_encoded[i][Y_train[i]]=1

  Y_val_encoded=np.zeros((Y_val.shape[0],10))
  for i in range(len(Y_val)):
    Y_val_encoded[i][Y_val[i]]=1

  Y_test_encoded=np.zeros((Y_test.shape[0],10))
  for i in range(len(Y_test)):
    Y_test_encoded[i][Y_test[i]]=1

  return X_train,X_test,X_val,Y_train_encoded,Y_val_encoded,Y_test_encoded

X_train,X_test,X_val,Y_train_encoded,Y_val_encoded,Y_test_encoded=dataset_preprocess()

Feedforward neural network class

In [44]:
#---------------------------------------------------------------ACTIVATION FUNCTIONS AND THEIR GRADIENTS-------------------------------------------------------------------------------
def relu(X):
  return max(X,0)

def grad_relu(X):
  return X>0

def sigmoid(X):
  return 1/(1+np.exp(-X))

def grad_sigmoid(X):
  return sigmoid(X)(1-sigmoid(X))

def tanh(X):
  return (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))

def grad_tanh(X):
  return 1-(tanh(X))**2

def softmax(X):
  # Calculates the softmax function
  e_X = np.exp(X - np.max(X, axis = 0))
  return e_X / e_X.sum(axis = 0)

def softmax_cross_entropy(Y,Y_pred):
  return -(Y - Y_pred) 



#-------------------------------------------------------------INITIALISATION FUNCTIONS------------------------------------------------------------------------------------------------
def random_initialisation(shape):
  # Initialising a random matrix with given dimensions (shape) as tuple
  np.random.seed(0)
  return np.random.randn(*shape)*0.1

def xavier_initialisation(shape):
    # Calculate the number of inputs and outputs
    n_in = shape[0]
    n_out = shape[1]
    
    # Calculate the variance of the normal distribution
    variance = 2.0 / (n_in + n_out)
    
    # Initialize the weights with random values drawn from the normal distribution
    np.random.seed(0)
    weights = np.random.randn(n_in, n_out) * np.sqrt(variance)
    
    return weights

#------------------------------------------------------------------INITIALIZING WEIGHTS AND BIASES------------------------------------------------------------------------------------
def initialize_w_b(input_layer,hidden_layer,output_layer,init):
  weights=[]
  biases=[]
  layers=[input_layer]+hidden_layer+[output_layer]
  for i in range(len(hidden_layer)+1):
     if init=='random':
       weights.append(random_initialisation((layers[i],layers[i+1])))
     if init=='xavier':
       weights.append(xavier_initialisation((layers[i],layers[i+1])))
     biases.append(np.random.randn(1,layers[i+1]))
  return weights, biases


#testing weights and biases output  
output_layer = 10
input_layer = X_test.shape[0]
hidden_layer = [64, 64]
weights, biases = initialize_w_b(input_layer, hidden_layer, output_layer, 'random')
for i in range(len(weights)):
    print(weights[i].shape)
    
for i in range(len(biases)):
    print(biases[i].shape)
print(weights[0][1])

(10000, 64)
(64, 64)
(64, 10)
(1, 64)
(1, 64)
(1, 10)
[ 0.01774261 -0.04017809 -0.16301983  0.04627823 -0.09072984  0.00519454
  0.07290906  0.01289829  0.11394007 -0.12348258  0.04023416 -0.06848101
 -0.08707971 -0.05788497 -0.03115525  0.00561653 -0.11651498  0.09008265
  0.04656624 -0.15362437  0.14882522  0.18958892  0.11787796 -0.01799248
 -0.10707526  0.10544517 -0.04031769  0.12224451  0.0208275   0.0976639
  0.03563664  0.07065732  0.00105     0.17858705  0.01269121  0.04019894
  0.18831507 -0.13477591 -0.1270485   0.09693967 -0.11731234  0.19436212
 -0.0413619  -0.07474548  0.1922942   0.14805148  0.1867559   0.09060447
 -0.08612257  0.1910065  -0.02680034  0.08024564  0.0947252  -0.01550101
  0.06140794  0.09222067  0.03764255 -0.10994008  0.02982382  0.13263859
 -0.06945679 -0.01496345 -0.04351536  0.18492637]


In [32]:
def random_initialisation(shape):
  # Initialising a random matrix with given dimensions (shape) as tuple
  np.random.seed(0)
  return np.random.randn(*shape)*0.1

print(random_initialisation((1,2)))

[[0.17640523 0.04001572]]
